In [1]:
%cd ../embeddings-for-trees

/home/anton/optimization-methods/embeddings-for-trees


In [2]:
import torch
import pickle

from data_module.jsonl_data_module import JsonlDataModule
from omegaconf import OmegaConf
import os

Using backend: pytorch


In [3]:
config = OmegaConf.load("config/tree_lstm_med_full.yaml")

In [4]:
data_module = JsonlDataModule(config)
data_module.prepare_data()
data_module.setup()

Dataset is already downloaded


In [5]:
id2label = data_module._vocabulary.id_to_label

In [6]:
id2label[0], id2label[1], id2label[2], id2label[3]

('<PAD>', '<UNK>', '<SOS>', '<EOS>')

In [7]:
pred_path = "/home/anton/opt-exps-data/outputs/full/Momentum_test_outputs.pkl"

In [8]:
id2label = data_module._vocabulary.id_to_label
test_dataloader = data_module.test_dataloader()

In [9]:
batch = next(iter(test_dataloader))

In [12]:
len(batch)

2

In [14]:
predictions = torch.load(pred_path, map_location=torch.device('cpu'))[0]

In [16]:
predictions.size()

torch.Size([8, 500])

In [18]:
test_labels = batch[0]

In [21]:
for batch_idx in range(test_labels.size(1)):
    preds, target = '', ''
    for idx in predictions[:, batch_idx]:
#         if idx == 2:
#             continue
#         if idx == 3:
#             break
        preds += f"{id2label[idx.item()]}|"
    for idx in test_labels[:, batch_idx]:
        target += f"{id2label[idx.item()]}|"
    print(f"PRED: {preds[:-1]}\nTARGET: {target[:-1]}\n")


PRED: <SOS>|get|id|<EOS>|<EOS>|<EOS>|<EOS>|<EOS>
TARGET: <SOS>|get|id|<EOS>|<PAD>|<PAD>|<PAD>|<PAD>

PRED: <SOS>|on|measure|<EOS>|changed|<EOS>|<EOS>|<EOS>
TARGET: <SOS>|on|measure|<EOS>|<PAD>|<PAD>|<PAD>|<PAD>

PRED: <SOS>|on|measure|<EOS>|changed|<EOS>|<EOS>|<EOS>
TARGET: <SOS>|on|measure|<EOS>|<PAD>|<PAD>|<PAD>|<PAD>

PRED: <SOS>|on|measure|<EOS>|changed|<EOS>|<EOS>|<EOS>
TARGET: <SOS>|on|measure|<EOS>|<PAD>|<PAD>|<PAD>|<PAD>

PRED: <SOS>|on|measure|<EOS>|changed|<EOS>|<EOS>|<EOS>
TARGET: <SOS>|on|measure|<EOS>|<PAD>|<PAD>|<PAD>|<PAD>

PRED: <SOS>|get|view|<EOS>|<EOS>|<EOS>|<EOS>|<EOS>
TARGET: <SOS>|get|view|<EOS>|<PAD>|<PAD>|<PAD>|<PAD>

PRED: <SOS>|create|<EOS>|<EOS>|<EOS>|<EOS>|<EOS>|<EOS>
TARGET: <SOS>|get|text|<EOS>|<PAD>|<PAD>|<PAD>|<PAD>

PRED: <SOS>|set|on|touch|listener|<EOS>|<EOS>|listener
TARGET: <SOS>|set|on|touch|listener|<EOS>|<PAD>|<PAD>

PRED: <SOS>|on|request|request|request|<EOS>|<EOS>|<EOS>
TARGET: <SOS>|on|dialog|item|click|<EOS>|<PAD>|<PAD>

PRED: <SOS>|get|tv|<

In [25]:
from utils.metrics import PredictionStatistic

In [26]:
statistic = PredictionStatistic(True, 0, [0,1,2,3])

In [28]:
statistic._mask_tensor_after_pad(predictions)[:, :10]

tensor([[   2,    2,    2,    2,    2,    2,    2,    2,    2,    2],
        [   4,    7,    7,    7,    7,    4,   11,    6,    7,    4],
        [  15,  562,  562,  562,  562,   48,    3,    7,   67, 3761],
        [   3,    3,    3,    3,    3,    3,    3,  397,   67,    3],
        [   3,   93,   93,   93,   93,    3,    3,  101,   67,    3],
        [   3,    3,    3,    3,    3,    3,    3,    3,    3,    3],
        [   3,    3,    3,    3,    3,    3,    3,    3,    3,    3],
        [   3,    3,    3,    3,    3,    3,    3,  101,    3,    3]])

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from optimizer import RLamb


model = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Sigmoid()
)

batch = torch.randn(100, 10)
labels = torch.randint(0, 1, (100, 1), dtype=torch.float)
optimizer = RLamb(model.parameters())

for epoch in range(10):
    pred = model(batch)
    loss = F.binary_cross_entropy(pred, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(epoch, loss.item())

0 0.520415723323822
1 0.5143498778343201
2 0.5111591219902039
3 0.5089250206947327
4 0.5071662664413452
5 0.5056905746459961
6 0.5056883096694946
7 0.505685567855835
8 0.5056823492050171
9 0.5056787729263306
